In [9]:
# =============================================================================
# SCRIPT DE ANÁLISIS COMPLETO DE MELANOMA (V13 - VERSIÓN SIMPLIFICADA)
# =============================================================================
# Este script usa solo Random Forest con las características seleccionadas
# y elimina todas las visualizaciones de gráficos.
# =============================================================================

# -----------------------------------------------------------------------------
# SECCIÓN 1: IMPORTACIONES Y CONFIGURACIÓN
# -----------------------------------------------------------------------------
import cv2
import numpy as np
import pandas as pd
import warnings

# Ignorar advertencias de Scikit-learn para una salida más limpia
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

print("INFO: Librerías cargadas correctamente.")

# -----------------------------------------------------------------------------
# SECCIÓN 2: PIPELINE FINAL Y EXTRACCIÓN DE CARACTERÍSTICAS
# -----------------------------------------------------------------------------

def obtener_mascara_final(img_color):
    """
    Implementa el pipeline final y robusto para obtener la máscara de un lunar.
    Toma una imagen a color y devuelve la máscara binaria final.
    """
    # 1. Corrección de Iluminación
    alto, ancho, _ = img_color.shape
    kernel_size = int(ancho * 0.25)
    if kernel_size % 2 == 0: kernel_size += 1
    fondo_estimado = cv2.GaussianBlur(img_color, (kernel_size, kernel_size), 0)
    img_corregida = cv2.divide(img_color, fondo_estimado, scale=255)

    # 2. Segmentación con Otsu
    gris_corregido = cv2.cvtColor(img_corregida, cv2.COLOR_BGR2GRAY)
    _, binarizada_otsu = cv2.threshold(gris_corregido, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 3. Limpieza Morfológica (Apertura)
    kernel_limpieza = np.ones((3, 3), np.uint8)
    mascara_limpia = cv2.morphologyEx(binarizada_otsu, cv2.MORPH_OPEN, kernel_limpieza, iterations=2)

    # 4. Post-procesamiento Final
    mascara_solo_lunar = np.zeros_like(mascara_limpia)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mascara_limpia, 4, cv2.CV_32S)
    if num_labels > 1:
        idx_max_area = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
        mascara_solo_lunar[labels == idx_max_area] = 255
        
    kernel_cierre = np.ones((15, 15), np.uint8)
    mascara_final = cv2.morphologyEx(mascara_solo_lunar, cv2.MORPH_CLOSE, kernel_cierre, iterations=2)
    
    return mascara_final

def calcular_features(imagen_color, mascara, feature_names_list):
    """Calcula un diccionario de características a partir de la imagen y su máscara."""
    features = {}
    # Si la máscara está vacía, devuelve ceros para evitar errores
    if np.sum(mascara) == 0: return {fname: 0 for fname in feature_names_list}

    contornos, _ = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos: return {fname: 0 for fname in feature_names_list}
    
    cnt = max(contornos, key=cv2.contourArea)
    M = cv2.moments(cnt)
    if M["m00"] > 0:
        centroide_x, centroide_y = M["m10"] / M["m00"], M["m01"] / M["m00"]
        distancias = np.sqrt(np.sum((cnt.squeeze() - [centroide_x, centroide_y])**2, axis=1))
        media_dist = distancias.mean()
        features['irregularidad_std_norm'] = np.std(distancias / media_dist) if media_dist > 0 else 0
    else:
        features['irregularidad_std_norm'] = 0

    media_bgr = cv2.mean(imagen_color, mask=mascara)
    features['color_r_promedio'], features['color_g_promedio'], features['color_b_promedio'] = media_bgr[2], media_bgr[1], media_bgr[0]
    
    return features

# -----------------------------------------------------------------------------
# SECCIÓN 3: CARGA DE DATOS Y CÁLCULO DE CARACTERÍSTICAS
# -----------------------------------------------------------------------------
print("\n--- Procesando todas las imágenes con el pipeline final ---")

# Solo las características seleccionadas
feature_names = ['irregularidad_std_norm', 'color_r_promedio', 'color_g_promedio']

# --- A. Procesar datos de entrenamiento ---
rutas_entrenamiento = [f'./files/mela{i}.jpeg' for i in range(1, 5)]
lista_features_train = []

for ruta in rutas_entrenamiento:
    img = cv2.imread(ruta)
    if img is None: continue
    
    # Recortar benignos y malignos
    benigno_img = img[57:, :104]
    maligno_img = img[57:, 110:]
    
    # Procesar benigno
    mascara_b = obtener_mascara_final(benigno_img)
    features_b = calcular_features(benigno_img, mascara_b, feature_names)
    features_b['clase'] = 'Benigno'
    lista_features_train.append(features_b)
    
    # Procesar maligno
    mascara_m = obtener_mascara_final(maligno_img)
    features_m = calcular_features(maligno_img, mascara_m, feature_names)
    features_m['clase'] = 'Maligno'
    lista_features_train.append(features_m)

df_train = pd.DataFrame(lista_features_train)

# --- B. Procesar datos de entrada ---
rutas_entrada = [f'./files/mela-entrada-{i}.jpg' for i in range(1, 4)]
lista_features_entrada = []

for ruta in rutas_entrada:
    img = cv2.imread(ruta)
    if img is None: continue
    mascara = obtener_mascara_final(img)
    features = calcular_features(img, mascara, feature_names)
    lista_features_entrada.append(features)

df_entrada = pd.DataFrame(lista_features_entrada)
print("INFO: Todas las características han sido calculadas.")

# -----------------------------------------------------------------------------
# SECCIÓN 4: CLASIFICACIÓN CON RANDOM FOREST
# -----------------------------------------------------------------------------
print("\n--- Entrenando modelo Random Forest ---")

X_train = df_train[feature_names]
y_train_labels = df_train['clase']
le = LabelEncoder()
y_train = le.fit_transform(y_train_labels)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Entrenar Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

print(f"Modelo: Random Forest")
print(f"Características utilizadas: {', '.join(feature_names)}")

# --- Clasificación Final ---
resultados = []
if not df_entrada.empty:
    X_new = df_entrada[feature_names]
    X_new_scaled = scaler.transform(X_new)
    probabilidades = clf.predict_proba(X_new_scaled)

    for i, prob in enumerate(probabilidades):
        prob_maligno = prob[1] # La clase 1 es 'Maligno'
        porcentaje = prob_maligno * 100
        categoria = "Benigno" if porcentaje <= 40 else "Inconcluso" if 41 <= porcentaje <= 70 else "Probablemente Maligno"
        resultados.append((rutas_entrada[i], categoria, porcentaje))

# -----------------------------------------------------------------------------
# SECCIÓN 5: RESULTADOS FINALES Y ADVERTENCIA
# -----------------------------------------------------------------------------
print("\n" + "="*50)
print("RESULTADOS DEL ANÁLISIS DE CLASIFICACIÓN")
print("="*50)

if resultados:
    for ruta, categoria, porcentaje in resultados:
        print(f"Archivo: {ruta}")
        print(f"  -> Veredicto: {categoria} ({porcentaje:.2f}% de probabilidad de malignidad)")
else:
    print("No se pudieron clasificar las imágenes de entrada.")

print("\n" + "-"*50)
print("ADVERTENCIA IMPORTANTE:")
print("Esta es una herramienta de software experimental y no un diagnóstico médico.")
print("Los resultados son una orientación basada en un conjunto de datos muy limitado.")
print("Consulte a un dermatólogo profesional para cualquier preocupación de salud.")
print("-"*50)

INFO: Librerías cargadas correctamente.

--- Procesando todas las imágenes con el pipeline final ---
INFO: Todas las características han sido calculadas.

--- Entrenando modelo Random Forest ---
Modelo: Random Forest
Características utilizadas: irregularidad_std_norm, color_r_promedio, color_g_promedio

RESULTADOS DEL ANÁLISIS DE CLASIFICACIÓN
Archivo: ./files/mela-entrada-1.jpg
  -> Veredicto: Benigno (17.00% de probabilidad de malignidad)
Archivo: ./files/mela-entrada-2.jpg
  -> Veredicto: Inconcluso (42.00% de probabilidad de malignidad)
Archivo: ./files/mela-entrada-3.jpg
  -> Veredicto: Benigno (13.00% de probabilidad de malignidad)

--------------------------------------------------
ADVERTENCIA IMPORTANTE:
Esta es una herramienta de software experimental y no un diagnóstico médico.
Los resultados son una orientación basada en un conjunto de datos muy limitado.
Consulte a un dermatólogo profesional para cualquier preocupación de salud.
---------------------------------------------